# Tagger 

This example shows the definition of a *tagger model* using simple `HashEmbed` and `Relu` layers.


In [148]:
from thinc.api import prefer_gpu
prefer_gpu()

from thinc.api import fix_random_seed
fix_random_seed(0)

from tqdm import tqdm



In [149]:
def train_model(data, model, optimizer, n_iter, batch_size):
    (train_X, train_y), (test_X, test_y) = data
    model.initialize(X=train_X[:5], Y=train_y[:5])
    for n in range(n_iter):
        loss = 0.0
        batches = model.ops.multibatch(batch_size, train_X, train_y, shuffle=True)
        for X, Y in tqdm(batches, leave=False):
            Yh, backprop = model.begin_update(X)
            d_loss = []
            for i in range(len(Yh)):
                d_loss.append(Yh[i] - Y[i])
                loss += ((Yh[i] - Y[i]) ** 2).sum()
            backprop(d_loss)
            model.finish_update(optimizer)
        score = evaluate(model, test_X, test_y, batch_size)
        print(f"{n}\t{loss:.2f}\t{score:.3f}")
        
def evaluate(model, test_X, test_Y, batch_size):
    correct = 0
    total = 0
    for X, Y in model.ops.multibatch(batch_size, test_X, test_Y):
        Yh = model.predict(X)
        for yh, y in zip(Yh, Y):
            correct += (y.argmax(axis=1) == yh.argmax(axis=1)).sum()
            total += y.shape[0]
    return float(correct / total)

In [150]:
from thinc.api import HashEmbed, Maxout, Softmax, expand_window, Relu
from thinc.api import residual, strings2arrays, with_array, clone, chain, concatenate


width = 32
n_tags = 17
vector_width = 16

model = chain(
    strings2arrays(),
    with_array(
        chain(
            HashEmbed(width, vector_width, column=0),
            expand_window(1),
            Relu(nI=96, nO=width),
            Relu(nI=width, nO=width),
            Softmax(nI=width, nO=n_tags)
        )
    )
)


In [152]:
import ml_datasets


CONFIG = """
[hyper_params]
width = 32
vector_width = 16
learn_rate = 0.001

[training]
n_iter = 10
batch_size = 128

[optimizer]
@optimizers = "Adam.v1"
learn_rate = ${hyper_params:learn_rate}
"""


from thinc.api import registry, Config

config = Config().from_str(CONFIG)
loaded_config = registry.make_from_config(config)

optimizer = loaded_config["optimizer"]
n_iter = loaded_config["training"]["n_iter"]
batch_size = loaded_config["training"]["batch_size"]

(train_X, train_y), (test_X, test_y) = data = ml_datasets.ud_ancora_pos_tags()



In [153]:
train_model(data, model, optimizer, n_iter, batch_size)

 12%|█▎        | 14/112 [00:00<00:00, 137.06it/s] 

0	392983.74	0.415


 12%|█▏        | 13/112 [00:00<00:00, 126.88it/s] 

1	300392.67	0.513


 12%|█▏        | 13/112 [00:00<00:00, 120.35it/s] 

...

 11%|█         | 12/112 [00:00<00:00, 117.51it/s] 

7	192651.78	0.675


 11%|█         | 12/112 [00:00<00:00, 117.24it/s] 

8	187864.93	0.680


9	183994.92	0.686
